In [1]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from tqdm import tqdm
from tad_mctc.data.molecules import mols as samples
from dxtb import Calculator
from dxtb.config import ConfigCache

dd: DD = {"dtype": torch.double, "device": torch.device("cuda:0")}
cache_config = ConfigCache(enabled=False, density=True, fock=True, overlap=True)
opts = {"verbosity": 2}

# Molecule
sample = samples["vancoh2"]
numbers = sample["numbers"].to(dd["device"])
print(f"len(numbers): {len(numbers)}")
positions = sample["positions"].clone().to(**dd)
charges = torch.tensor(0.0, **dd)

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
# opts = dict(opts, **{"scf_mode": "reconnect", "scp_mode": "potential"})
opts = dict(opts, **{"scf_mode": "implicit", "scp_mode": "charges"})
# opts = dict(opts, **{"scf_mode": "reconnect"})

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd, timer=True)
dxtb.timer.reset()
calc.opts.cache = cache_config

print(f"Calc scf mode: {calc.opts.scf.scf_mode}")
print(f"Calc scp mode: {calc.opts.scf.scp_mode}")

charges = calc.get_charges(pos, charges, scf_charges_guess=None)
print(f"Charges.shape: {charges.shape}")
dxtb.timer.print(v=1)

len(numbers): 176
Calc scf mode: 1
Calc scp mode: 1
SCF Energy  : -326.18651248195351 Hartree.
Total Energy: -322.95378788487272 Hartree.
Charges.shape: torch.Size([550])


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.040           5.62
 - Repulsion                0.026          64.81
 - DispersionD3             0.013          31.48
 - Halogen                  0.001           3.48
Integrals                   0.063           8.78
 - Overlap                  0.060          94.54
 - Core Hamiltonian         0.003           5.44
Intor                       0.031           4.27
ovlp.norm                   0.020           2.75
SCF                         0.543          75.67
 - Interaction Cache        0.001           0.19
 - Potential                0.467          86.07
 - Fock build               0.001           0.13
 - Diagonalize              0.428          78.81
 - Density                

# Reconnect vs implicit testing

### Energy and forces

In [2]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from tqdm import tqdm
from tad_mctc.data.molecules import mols as samples
from dxtb import Calculator
from dxtb.config import ConfigCache

# dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}
cache_config = ConfigCache(enabled=False, density=True, fock=True)
opts = {"verbosity": 2}

# Molecule
sample = samples["vancoh2"]
numbers = sample["numbers"].to(dd["device"])
positions = sample["positions"].clone().to(**dd)
charges = torch.tensor(0.0, **dd)

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
opts = dict(opts, **{"scf_mode": "implicit", "scp_mode": "charges"})

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd, timer=True)
dxtb.timer.reset()
calc.opts.cache = cache_config

e_implicit = calc.get_energy(pos, charges)
scf_charges = calc.get_charges(pos, charges, scf_charges_guess=None)
dxtb.timer.print(v=1)

SCF Energy  : -326.18651248195357 Hartree.
Total Energy: -322.95378788487278 Hartree.
SCF Energy  : -326.18651248195357 Hartree.
Total Energy: -322.95378788487278 Hartree.


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.008           0.88
 - Repulsion                0.001          15.28
 - Halogen                  0.000           3.57
 - DispersionD3             0.007          80.03
Integrals                   0.026           2.79
 - Overlap                  0.022          84.79
 - Core Hamiltonian         0.004          15.15
SCF                         0.891          95.97
 - Interaction Cache        0.001           0.12
 - Potential                0.848          95.11
 - Fock build               0.001           0.16
 - Diagonalize              0.791          88.77
 - Density                  0.025           2.81
 - Charges                  0.006           0.68
-------------------------

In [4]:
opts = dict(opts, **{"scf_mode": "reconnect", "scp_mode": "charges"})
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd, timer=True)
dxtb.timer.reset()
calc.opts.cache = cache_config

e_reconnect = calc.get_energy(pos, charges, scf_charges_guess=scf_charges)
dxtb.timer.print(v=1)

SCF Energy  : -326.18651247956814 Hartree.
Total Energy: -322.95378788248729 Hartree.


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.008          12.50
 - Halogen                  0.000           3.33
 - DispersionD3             0.006          82.87
 - Repulsion                0.001          12.96
Integrals                   0.023          38.45
 - Overlap                  0.020          85.57
 - Core Hamiltonian         0.003          14.38
SCF                         0.027          45.08
 - Interaction Cache        0.001           4.77
 - Potential                0.023          83.64
 - Fock build               0.000           0.23
 - Diagonalize              0.021          77.00
 - Density                  0.001           2.54
 - Charges                  0.000           0.77
------------------------------------------------
Sum                         0.058          96.04
Total        

In [6]:
f_implicit = torch.autograd.grad(e_implicit, pos, retain_graph=True)[0]
f_reconnect = torch.autograd.grad(e_reconnect, pos, retain_graph=True)[0]

print(f"e_implicit: {e_implicit}, e_reconnect: {e_reconnect}")
# print(f"f_implicit: {f_implicit}")
# print(f"f_reconnect: {f_reconnect}")
print(f"max diff: {torch.max(torch.abs(f_implicit - f_reconnect))}")

e_implicit: -322.9537878848728, e_reconnect: -322.9537878824873
max diff: 3.282661192043068e-05


### Torch.no_grad()

In [ ]:
# Molecule
sample = samples["vancoh2"]
numbers = sample["numbers"].to(dd["device"])
positions = sample["positions"].clone().to(**dd)
charges = torch.tensor(0.0, **dd)

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
opts = dict(opts, **{"scf_mode": "implicit", "scp_mode": "charges"})

dxtb.timer.reset()
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd)
calc.opts.cache = cache_config

with torch.no_grad():
    e_implicit = calc.get_energy(pos, charges)
    scf_charges = calc.get_charges(pos, charges, scf_charges_guess=None)
    dxtb.timer.print(v=1)

In [ ]:
opts = dict(opts, **{"scf_mode": "reconnect", "scp_mode": "charges"})

pos = positions.clone().requires_grad_(True)
dxtb.timer.reset()

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd)
calc.opts.cache = cache_config

e_reconnect = calc.get_energy(pos, charges, scf_charges_guess=scf_charges)
dxtb.timer.print(v=1)

In [ ]:
# f_implicit = torch.autograd.grad(e_implicit, pos, retain_graph=True)[0]
f_reconnect = torch.autograd.grad(e_reconnect, pos, retain_graph=True)[0]

print(f"e_implicit: {e_implicit}, e_reconnect: {e_reconnect}")
# print(f"f_implicit: {f_implicit}")
# print(f"f_reconnect: {f_reconnect}")
print(f"max diff: {torch.max(torch.abs(f_implicit - f_reconnect))}")